In [1]:
# Copyright 2021 Google
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Time Crystal Circuit Generation
This notebook covers how Many Body Local Discrete Time Crystal circuit lists are created, from the paper: Observation of Time-Crystalline Eigenstate Order on a Quantum Processor ([Nature](https://www.nature.com/articles/s41586-021-04257-w)). 

Quantum computers and gate-based quantum circuits turn out to be well suited for crafting systems that exhibit time-crystalline behavior. Behavior is crystalline with respect to time if it has some consistent and stable pattern over time. This system's pattern must be resilient against perturbation in the same way that a space-crystalline object, like a diamond, is still a diamond if moved or heated. 

The quantum computer supplies a system of many qubits, locally connected to each other in a chain. A many-body local system like this is critical for the existence of a time crystal. Without an MBL system, it is expected that the system's state would decay into a maximum entropy state that is incompatible with the goal of stable and consistent time structure. 

The time-crystalline behavior that the DTC circuits demonstrate is perhaps the simplest kind of time-structured behavior, oscillation. Each circuit is built with some number of identical $U$-cycles. Time is represented by a circuit list where each circuit is ordered with increasingly many $U$-cycles; each cycle is a discrete time step. The eventual effect of these $U$-cycles is consistent oscillations of each qubits' polarizations. The experiments performed demonstrate that this time-crystalline oscillation behavior is stable in spite of different initial states and introduced random potentials. 

## Setup

In [2]:
!pip install cirq --pre --quiet
try:
    import recirq
except ImportError:
    !pip install --quiet git+https://github.com/quantumlib/ReCirq

In [3]:
import cirq
import recirq.time_crystals as time_crystals

## Circuit Construction
Each DTC circuit is created with symbolic parameters. Parameter values are supplied near run/simulation time with a `cirq.ParamResolver`, which means the circuit list needs to be generated only once for potentially many different experimental parameter configurations. 

The code below uses an IPython-specific utility to inspect the code of the key function that creates the symbolic circuit list, `recirq.time_crystals.symbolic_dtc_circuit_list()`. 

In [4]:
import inspect
from IPython.display import Code
Code(inspect.getsource(time_crystals.symbolic_dtc_circuit_list), language='python')

def symbolic_dtc_circuit_list(
        qubits: Sequence[cirq.Qid],
        cycles: int
        ) -> List[cirq.Circuit]:

    """ Create a list of symbolically parameterized dtc circuits, with increasing cycles
    Args:
        qubits: ordered sequence of available qubits, which are connected in a chain
        cycles: maximum number of cycles to generate up to
    Returns:
        list of circuits with `0, 1, 2, ... cycles` many cycles
    """

    num_qubits = len(qubits)

    # Symbol for g
    g_value = sp.Symbol('g')

    # Symbols for random variance (h) and initial state, one per qubit
    local_fields = sp.symbols('local_field_:' + str(num_qubits))
    initial_state = sp.symbols('initial_state_:' + str(num_qubits))

    # Symbols used for PhasedFsimGate, one for every qubit pair in the chain
    thetas = sp.symbols('theta_:' + str(num_qubits - 1))
    zetas = sp.symbols('zeta_:' + str(num_qubits - 1))
    chis = sp.symbols('chi_:' + str(num_qubits - 1))
    gammas = sp.symbols('gamma_:' + str(num_qubits - 1))
    phis = sp.symbols('phi_:' + str(num_qubits - 1))

    # Initial moment of Y gates, conditioned on initial state
    initial_operations = cirq.Moment([cirq.Y(qubit) ** initial_state[index] for index, qubit in enumerate(qubits)])

    # First component of U cycle, a moment of XZ gates.
    sequence_operations = []
    for index, qubit in enumerate(qubits):
        sequence_operations.append(cirq.PhasedXZGate(
                x_exponent=g_value, axis_phase_exponent=0.0,
                z_exponent=local_fields[index])(qubit))

    # Initialize U cycle
    u_cycle = [cirq.Moment(sequence_operations)]

    # Second and third components of U cycle, a chain of 2-qubit PhasedFSim gates
    #   The first component is all the 2-qubit PhasedFSim gates starting on even qubits
    #   The second component is the 2-qubit gates starting on odd qubits
    operation_list, other_operation_list = [],[]
    previous_qubit, previous_index = None, None
    for index, qubit in enumerate(qubits):
        if previous_qubit is None:
            previous_qubit, previous_index = qubit, index
            continue

        # Add an fsim gate
        coupling_gate = cirq.ops.PhasedFSimGate(
            theta=thetas[previous_index],
            zeta=zetas[previous_index],
            chi=chis[previous_index],
            gamma=gammas[previous_index],
            phi=phis[previous_index]
        )
        operation_list.append(coupling_gate.on(previous_qubit, qubit))

        # Swap the operation lists, to avoid two-qubit gate overlap
        previous_qubit, previous_index = qubit, index
        operation_list, other_operation_list = other_operation_list, operation_list

    # Add the two components into the U cycle
    u_cycle.append(cirq.Moment(operation_list))
    u_cycle.append(cirq.Moment(other_operation_list))

    # Prepare a list of circuits, with n=0,1,2,3 ... cycles many cycles
    circuit_list = []
    total_circuit = cirq.Circuit(initial_operations)
    circuit_list.append(total_circuit.copy())
    for c in range(cycles):
        for m in u_cycle:
            total_circuit.append(m)
        circuit_list.append(total_circuit.copy())

    return circuit_list

The construction of each circuit is surprisingly succinct. 

The circuit expects the quantum computer to be in the all-zeros state, and starts with a sequence of `cirq.Y` gates conditioned on the provided `initial state` parameter, after initializing the necessary symbolic variables. 

Each $U$-cycle consists of three moments. First, a moment of `cirq.PhasedXZGate`s, with one for each qubit. Each `cirq.PhasedXZGate` takes the control parameter `g` as its X-exponent, and the random potentials necessary for many-body localization provided by `local_fields` for its Y-exponent.

The second and third moments both cause the oscillation behavior and compensate for the first disorder moment. The qubits are connected in a chain, and each qubit pair connection in that chain is coupled with a `cirq.PhasedFSimGate` that uses the parameters `[theta, zetas, chi, gamma, phi]`. To keep gates from overlapping on the same qubit, this chain of gates is split into the second and third moments, such that no two gates share a qubit within each moment. 

Finally, `symbolic_dtc_circuit_list()` builds and returns a list of circuits with $0,1,2,..., cycles$ many $U$-cycles in them. 

In [5]:
qubits = [cirq.GridQubit(0,i) for i in range(4)]
circuit_list = time_crystals.symbolic_dtc_circuit_list(qubits, 2)
for circuit in circuit_list: 
    print('\ncircuit of length ' + str(len(circuit)) + "\n")
    print(circuit)


circuit of length 1

(0, 0): ───Y^initial_state_0───

(0, 1): ───Y^initial_state_1───

(0, 2): ───Y^initial_state_2───

(0, 3): ───Y^initial_state_3───

circuit of length 4

(0, 0): ───Y^initial_state_0───PhXZ(a=0,x=g,z=local_field_0)────────────────────────────────────────────────────PhFSim(theta_0, zeta_0, chi_0, gamma_0, phi_0)───
                                                                                                                │
(0, 1): ───Y^initial_state_1───PhXZ(a=0,x=g,z=local_field_1)───PhFSim(theta_1, zeta_1, chi_1, gamma_1, phi_1)───PhFSim(theta_0, zeta_0, chi_0, gamma_0, phi_0)───
                                                               │
(0, 2): ───Y^initial_state_2───PhXZ(a=0,x=g,z=local_field_2)───PhFSim(theta_1, zeta_1, chi_1, gamma_1, phi_1)───PhFSim(theta_2, zeta_2, chi_2, gamma_2, phi_2)───
                                                                                                                │
(0, 3): ───Y^initial_state_3───PhXZ(a=0,x=g,z=

After the initial line of `cirq.Y` gates, each consecutive circuit in the list has an additional cycle of `cirq.PhasedXZGate`s, followed by the chain of `cirq.PhasedFSimGate`s on alternating qubit pairs. Each cycle of three moments becomes one time step in the later analysis of stable oscillations over time. 

The next step is to perform experiments to collect evidence of the time-crystalline behavior of the quantum state's polarizations. See the [Time Crystal Data Collection](time_crystal_data_collection.ipynb) notebook for the experiments, and the [Time Crystal Data Analysis](time_crystal_data_analysis.ipynb) notebook for the graphed data and results. 